In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import log
import torch.optim as optim
import time
from WAC import WAC
from data_pre import data_preprocessing
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader


torch.manual_seed(1)
random.seed(1)

print(torch.cuda.is_available())
print(torch.__version__)

'''
TO DO: 
* Use batching
    * set grad to be 0
    * set loss to be 0
    * loss.backward() and optimizer
'''

'''
load and prepare data
'''
(voc_ix, trainX, trainy, testX, testy, devX, devy) = data_preprocessing() 
print("finish preparing data\n")



False
1.0.1.post2
data preprocessing takes 1.112109899520874
finish preparing data



In [28]:
trainX = torch.stack(trainX)

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 7 and 12 in dimension 2 at /Users/soumith/mc3build/conda-bld/pytorch_1549593514549/work/aten/src/TH/generic/THTensorMoreMath.cpp:1307

In [21]:
ds = DataLoader((trainX, trainy), batch_size=10)

In [23]:
for i,(X,y) in enumerate(ds):
    print(i)

RuntimeError: Expected object of scalar type Long but got scalar type Float for sequence elment 1 in sequence argument at position #1 'tensors'

In [ ]:
'''
set parameters
'''          
## set hyperparameters 
VOCAB_SIZE = len(voc_ix)
EMBEDDING_DIM = 100
eval_per = 5000
n_epoch = 5
bsize = 500
PATH = "../model/wac.pt"

## define model 
model = WAC(EMBEDDING_DIM, VOCAB_SIZE)
optimizer = optim.Adagrad(model.parameters(), lr = 1e-2, lr_decay = 1e-4)


'''
training
'''
## training 
losses = []
accs = []
i = 0
best_dev_acc = 0
start = time.time()
for epoch in range(n_epoch):  
    print("epoch " + str(epoch))
    total_loss = 0
    loss = 0

    ds = DataLoader((trainX, trainy), batch_size=10, shuffle=True)
    #(trainX, trainy) = ds.dataset

    for X,y in enumerate(ds):

        if i % eval_per == 0:
            print("time: {}".format(time.time() - start))
            acc = model.evaluate(devX, devy)
            if acc > best_dev_acc:
                best_dev_acc = acc
                torch.save(model, PATH)
            print("accuracy on dev: " + str(acc))
            accs.append(acc)
        
        # Step 3. Run our forward pass.
        prob = model.forward(X)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss_sent = - y*log(prob) - (1-y)*log(1-prob) 
        loss += loss_sent

        if i % bsize == 0:
            loss.backward()
            optimizer.step()
            model.zero_grad()
            total_loss += loss.item()
            loss = 0
        i +=1
        
    losses.append(total_loss)
runtime = time.time() - start
print("runtime: " + str(runtime) + "s")

model_best = torch.load(PATH)
model_best.eval()
acc_dev = model_best.evaluate(devX, devy)
print("best model acc on dev: " + str(acc_dev))
acc_test = model_best.evaluate(testX, testy)
print("best model acc on test: " + str(acc_test))


